In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('data.csv',sep=';',dtype=str)

In [ ]:
colunas = [i for i in data.columns if 'Unnamed' not in i]

In [ ]:
data = data[colunas]
data = data.fillna('-1')

In [ ]:
data = data[data['Você sofreu uma nova lesão do ligamento cruzado anterior?']!='-1']

In [ ]:
data.head(5)

In [ ]:
for c in data.columns:
    try:
        data[c] = data[c].str.replace(',','.')
    except Exception as e:
        print(e)

In [ ]:
numeric_data = data.apply(pd.to_numeric, errors='coerce')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = numeric_data.corr()
plot = sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)
figure = plot.get_figure()    
figure.savefig('correlation.png', dpi=400)

In [ ]:
pd.options.display.max_columns = None

In [ ]:
X = numeric_data.drop(labels=['Você sofreu uma nova lesão do ligamento cruzado anterior?'],
                      axis=1)
y = numeric_data['Você sofreu uma nova lesão do ligamento cruzado anterior?']

In [ ]:
categorical_cols = ['Sexo',
                    'Dominância',
                    'Membro Lesionado',
                    'Lesões Associadas',
                    'Tipo de Enxerto',
                    'Prática Esportiva',
                    'Retorno ao Esporte',
                    'Pretende Retornar',
                    'Lesão contralateral ou no enxerto?',
                    'Você retornou a praticar o mesmo esporte que praticava antes da primeira lesão?',
                    'Qual o esporte que você praticava antes da lesão?',
                    'Retornou ao mesmo nível de antes da lesão? ',
                    'Se não para a pergunta anterior, tem intenção de retornar?',
                    'Você pratica alguma atividade física hoje em dia?',
                    'Tegner']
numerical_cols = [x for x in X.columns if x
                    not in categorical_cols]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.7)

In [ ]:
mm = MinMaxScaler()
X_train_num = X_train[numerical_cols]
X_train_num = mm.fit_transform(X_train_num)
X_train = np.concatenate([X_train_num,X_train[categorical_cols]],axis=1)

X_test_num = X_test[numerical_cols]
X_test_num = mm.transform(X_test_num)
X_test = np.concatenate([X_test_num,X_test[categorical_cols]],axis=1)

In [ ]:
smote = SMOTE(sampling_strategy='auto')
X_train, y_train = smote.fit_sample(X_train, y_train)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=X_train.shape[1]-28)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
dt.score(X_test,y_test)

In [ ]:
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_train,y_train)
rf.score(X_test,y_test)

In [ ]:
svm = SVC(kernel='rbf',degree=3,C=2.0)
svm.fit(X_train,y_train)
svm.score(X_test,y_test)

In [ ]:
nn = MLPClassifier(hidden_layer_sizes=[10,10,10],activation='relu',max_iter=400)
nn.fit(X_train,y_train)
nn.score(X_test,y_test)

In [ ]:
print(classification_report(y_test,dt.predict(X_test)))

In [ ]:
print(classification_report(y_test,rf.predict(X_test)))

In [ ]:
print(classification_report(y_test,svm.predict(X_test)))

In [ ]:
print(classification_report(y_test,nn.predict(X_test)))

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
plot_data = plot_tree(dt)